# DSCI 525 Group 4 - Data retrieval using figshare API

## Import libraries:

In [1]:
import re
import os
import sys
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import timeit

## Specify Meta variables

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "..\data"
file_to_download = "data.zip"
rerun = True

## List of files available for download

In [3]:
start = timeit.default_timer()
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
print(f"Run time: {str(datetime.timedelta(minutes=timeit.default_timer()-start))}.")
files

Run time: 0:00:45.950148.


[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

## Download specified file

In [4]:
start = timeit.default_timer()
if os.path.exists(f"{output_directory}\\{file_to_download}"):
    print("data.zip is already exists!")
else:
    files_to_dl = ["data.zip"]

    for file in files:
        if file["name"] in files_to_dl:
            os.makedirs(output_directory, exist_ok=True)
            urlretrieve(file["download_url"], os.path.join(output_directory, file["name"]))
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

data.zip is already exists!
Run time: 0:00:00.


## Unzip downloaded file

In [5]:
start = timeit.default_timer()
if rerun:
    with zipfile.ZipFile(os.path.join(output_directory, file_to_download), 'r') as f:
        f.extractall(output_directory)
else:
    print("Some CSV files already exists, nothing is extraced. Please check if files in data directory are correct.")
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

Run time: 0:15:00.


## 4. Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.

2. When combining the CSV files, add an extra column called "model" that identifies the model. Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON Tip 2: Remember how we added year when we combined airline CSVs. Tip 3: You can use regex generator.

Note: There is a file called observed_daily_rainfall_SYD.csv in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from folder) before you combine CSVs. We will use this file in our next milestone.

3. Compare run times on different machines within your team and summarize your observations.

In [6]:
start = timeit.default_timer()
if rerun:
    files = glob.glob(f'{output_directory}/*.csv')
    files = [f for f in files if f.find("observed_daily_rainfall_SYD.csv")==-1 and f.find("combined_data.csv")==-1]
    
    if len(files) != 27:
        print("Expected 27 files.")
        
    i = 1
    records = 0

    for file in files:
        df = pd.read_csv(file, index_col=0, parse_dates=True).assign(model=re.findall(r'[^\/&\\]+(?=_daily_rainfall_NSW\.)', file)[0])
        print(f"Processing {file:<60} total {len(df)}{' rows, ':<25}{i} out of {len(files)} files.")
        records += len(df)

        if i == 1:
            df.to_csv(f"{output_directory}/combined_data.csv")
        else:
            df.to_csv(f'{output_directory}/combined_data.csv', mode='a', header=False)

        i+=1
    print("")
    print(f"Total rows: {records}.") #62467843 rows
    print("")
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

Processing ..\data\ACCESS-CM2_daily_rainfall_NSW.csv                    total 1932840 rows,                   1 out of 27 files.
Processing ..\data\ACCESS-ESM1-5_daily_rainfall_NSW.csv                 total 1610700 rows,                   2 out of 27 files.
Processing ..\data\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv                total 966420 rows,                   3 out of 27 files.
Processing ..\data\BCC-CSM2-MR_daily_rainfall_NSW.csv                   total 3035340 rows,                   4 out of 27 files.
Processing ..\data\BCC-ESM1_daily_rainfall_NSW.csv                      total 551880 rows,                   5 out of 27 files.
Processing ..\data\CanESM5_daily_rainfall_NSW.csv                       total 551880 rows,                   6 out of 27 files.
Processing ..\data\CMCC-CM2-HR4_daily_rainfall_NSW.csv                  total 3541230 rows,                   7 out of 27 files.
Processing ..\data\CMCC-CM2-SR5_daily_rainfall_NSW.csv                  total 3541230 rows,         

## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 7min 25s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 10min 26s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes | 7min 22s |
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 6 mins 34s |

In [7]:
%%sh
du -sh "../data/combined_data.csv"

5.7G	../data/combined_data.csv


## 5. Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).

- Changing dtype of your data
- Load just columns what we want
- Loading in chunks
- Dask
2. Compare run times on different machines within your team and summarize your observations.

## Benchmark: Load everything

In [8]:
start = timeit.default_timer()
df = pd.read_csv(f"{output_directory}/combined_data.csv", index_col=0, parse_dates=True)
print("Output:")
print("")
print(df["model"].value_counts())
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")
print(f"The size of df in memory: {round(sys.getsizeof(df) / (1024 * 1024 * 1024), 2)} GB.")

Output:

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
--------------------------------------------------
Run time: 1:31:00.
The size of df in memory: 6.72 GB.


## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 1min 15s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 2min 59s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes |1min 30s |
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 1 mins 31s |

## Change dtype of data

In [9]:
start = timeit.default_timer()
dtypes = {'lat_min': 'float16', 
          'lat_max': 'float16', 
          'lon_min': 'float16', 
          'lon_max': 'float16',
          'rain (mm/day)':'float32',
          'model':'str'}
df = pd.read_csv(f"{output_directory}/combined_data.csv",index_col=0, parse_dates=True, dtype=dtypes)
print("Output:")
print("")
print(df["model"].value_counts())
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")
print(f"The size of df in memory: {round(sys.getsizeof(df) / (1024 * 1024 * 1024), 2)} GB.")
print("")
print(f"Column types:")
df.dtypes

Output:

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
--------------------------------------------------
Run time: 1:27:00.
The size of df in memory: 5.09 GB.

Column types:


lat_min          float16
lat_max          float16
lon_min          float16
lon_max          float16
rain (mm/day)    float32
model             object
dtype: object

## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 1min 12s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 2min 7s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes | 1min 35s |
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 1 mins 27s |

## Load only minimum columns

In [10]:
start = timeit.default_timer()
use_cols = ["time","rain (mm/day)","model"]
df = pd.read_csv(f"{output_directory}/combined_data.csv",index_col=0, parse_dates=True, usecols=use_cols)
print("Output:")
print("")
print(df["model"].value_counts())
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")
print(f"The size of df in memory: {round(sys.getsizeof(df) / (1024 * 1024 * 1024), 2)} GB.")

Output:

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
--------------------------------------------------
Run time: 1:16:00.
The size of df in memory: 4.86 GB.


## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 1min 5s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 1min 32s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes |1min 23s|
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 1 mins 16s |

## Load only minimum columns and specify column types

In [11]:
start = timeit.default_timer()
use_cols = ["time","rain (mm/day)","model"]
dtypes = {'rain (mm/day)':'float32',
          'model':'str'}
df = pd.read_csv(f"{output_directory}/combined_data.csv",index_col=0, parse_dates=True, usecols=use_cols,dtype=dtypes)
print("Output:")
print("")
print(df["model"].value_counts())
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")
print(f"The size of df in memory: {round(sys.getsizeof(df) / (1024 * 1024 * 1024), 2)} GB.")

Output:

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
--------------------------------------------------
Run time: 1:16:00.
The size of df in memory: 4.62 GB.


## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 1min 22s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 1min 51s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes | 1min 22s |
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 1 mins 16s |

## Loading in chunks (1 million rows)

In [12]:
start = timeit.default_timer()
counts = pd.Series(dtype=int)

df = pd.read_csv(f"{output_directory}/combined_data.csv", chunksize=1_000_000 )

for chunk in pd.read_csv(f"{output_directory}/combined_data.csv", chunksize=1_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)

print("Output:")
print("")
print(counts.astype(int))
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

Output:

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
--------------------------------------------------
Run time: 0:57:00.


## Compare results:

| Team Member          | Operating System | RAM (GB) | Processor                 | Is SSD | Time taken |
| -------------------- | ---------------- | -------- | ------------------------- | ------ | ---------- |
| Anahita Einolghozati | MacBook Pro| 8 | M1 | Yes| 48.9s |
| Luke Collins         | Windows 11 x64 | 16 | 11th Gen Intel i7 2.80 GHz | Yes | 1min 21s |
| Zihan Zhou           | MacBook Pro  |  8 | M1 | Yes | 1min 11s |
| Steven Lio           | Windows 10 x64   | 16     | AMD Ryzen 7 5800H 3.20GHz | Yes    | 57s        |

# Dicussion

Compares to native pandas csv import:

Change dtype of data:
- Pros: reduce size in memory after reading
- Cons: best data type is not usually known ahead of time

Load only minimum columns:
- Pros: faster import and reduce size in memory after reading
- Cons: if new column is needed the file has to be import again

Load only minimum columns & specify dtype:
- Pros: faster import and further reduce size in memory after reading
- Cons: if new column is needed the file has to be import again and best data type is not usually known ahead of time

Loading in chunks:
- Pros: fastest in import and uses much less memory if the chunks are not being told to store in memory
- Cons: only usable if program can work with chunks of data (say if we do mapping of data points then chunks won't work)

## Compare dataframe transfer from Python to R for EDA using Parquet file format

Import full dataframe to Python:

In [13]:
if rerun:
    df = pd.read_csv(f"{output_directory}/combined_data.csv",index_col=0, parse_dates=True)

Convert data file into parquet format:

In [14]:
start = timeit.default_timer()
if rerun:
    df.to_parquet(f"{output_directory}/combined_data.parquet")
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

Run time: 0:14:00.


Convert data file into parquet format (with partition by model):

In [15]:
start = timeit.default_timer()
if rerun:
    df.to_parquet(f"{output_directory}/combined_data_partition.parquet",partition_cols=['model'])
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")

Run time: 0:23:00.


Compare storage size in drive:

In [16]:
%%sh
du -sh "../data/combined_data.csv"
du -sh "../data/combined_data.parquet"
du -sh "../data/combined_data_partition.parquet"

5.7G	../data/combined_data.csv
542M	../data/combined_data.parquet
550M	../data/combined_data_partition.parquet


Import parquet file to Python and row count by model:

In [17]:
start = timeit.default_timer()
df = pd.read_parquet(f"{output_directory}/combined_data.parquet")
print("Output:")
print("")
print(df["model"].value_counts())
print("-"*50)
print(f"Run time: {str(datetime.timedelta(minutes=int(timeit.default_timer()-start)))}.")
print(f"The size of df in memory: {round(sys.getsizeof(df) / (1024 * 1024 * 1024), 2)} GB.")

Output:

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
--------------------------------------------------
Run time: 0:08:00.
The size of df in memory: 6.72 GB.


Import into R and row count by model:

In [18]:
%load_ext rpy2.ipython

C:\Users\kpgha\.conda\envs\525_2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [19]:
%%R

suppressPackageStartupMessages(library(dplyr,quietly=TRUE))

start <- Sys.time()
output_directory <- "../data"
#df <- arrow::open_dataset(paste0(output_directory,"/","combined_data.parquet"))
df <- arrow::open_dataset(paste0(output_directory,"/","combined_data_partition.parquet"), 
                          format="parquet", 
                          partitioning=c("model"))
df %>%
    group_by(model) %>%
    summarize(cnt=n()) %>%
    ungroup() %>%
    collect()

Sys.time() - start

Time difference of 1.867574 secs


R is much more impressive in opening parquet file than Python in terms of I/O speed. (1.9s vs 8s)

## Discussion:

Our team explored all 4 dataframe transfer methods and we picked `parquet` for this following reasons:
- The implementation of converting to `parquet` is already provided in pandas. 
- Although `feather` is also supported by pandas but `parquet` has the better compression than `feather` where the storage memory for `parquet` of this data set is almost half of `feather`. 
- We found that pandas' implementation converting to `feather` does not support pandas dataframe with a datetime index which we have in this case, where `parquet` does not care about the datetime index as much, although it did took slightly longer time to write into `parquet` format.
- `parquet` has best support in terms of data partitioning, also it can have much faster read when subset of the columns, data were selected given the projection and predicate pushdown capabilities provided by the arrow engine
- `feather` was design to transfer data between Python and R but we also consider `parquet` is better for long term storage (perfect for historical data like this one) and we found no significant time differences between these two file types.

We also looked at:
- Pandas Exchange: was not able to complete since the conversion took too much memory (RAM) and we were unable to run.
- Pyarrow: we found no easy way for R to read in a arrow file type direct from drive and direct conversion is fast but implementation is more complicated than panda's built-in method

Overall converting data to these types provide the benefits for efficient storage, I/O time and providing convenient way of transferring data between Python and R.